## Тематическая модель на данных Last.fm

Сначала настраиваем и подключаем нужные библиотеки

In [22]:
import os
import sys
HOME = '/home/vovapolu/Projects/'
BIGARTM_PATH = HOME + 'bigartm/'
BIGARTM_BUILD_PATH = BIGARTM_PATH + 'build/'
sys.path.append(os.path.join(BIGARTM_PATH, 'src/python'))
os.environ['ARTM_SHARED_LIBRARY'] = os.path.join(BIGARTM_BUILD_PATH, 'src/artm/libartm.so')

In [31]:
import artm.messages_pb2, artm.library, uuid
import csv

plays_file = 'usersha1-artmbid-artname-plays.tsv'
batch_path = 'batches' #Папка с батчами

artist_id_to_name = {} #Мапа, переводящая artist_id в имя
artists_idxs = {} #Мапа, переводящая artist_id в номер в батче
artists = [] #Имена артистов в батче

Читаем файл и строим батчи

In [29]:
last_user_id = ''
handled_users = 0

users_to_handle = 20000 #Данные скольких юзеров обрабатываем
users_in_batch = 1000 #Сколько юзеров в батче

batch = None

with open(plays_file, 'rb') as tsvin:
    tsvin = csv.reader(tsvin, delimiter='\t')

    field = None

    for row in tsvin:
        
        user_id, artist_id, artist_name, plays = row

        if user_id != last_user_id:
            if handled_users > users_to_handle or handled_users % users_in_batch == 0:
                if batch is not None:
                    for artist in artists:
                        batch.token.append(artist.decode('utf8'))
                    artm.library.Library().SaveBatch(batch, batch_path)
                    artists = []
                    artists_idxs = {}
                batch = artm.messages_pb2.Batch()
                batch.id = str(uuid.uuid4())
            
            if handled_users > users_to_handle:
                break
                        
            item = batch.item.add()
            item.id = handled_users
            field = item.field.add()
            
            last_user_id = user_id
            handled_users += 1
            
        if artist_id not in artist_id_to_name:
            artist_id_to_name[artist_id] = artist_name
        if artist_id not in artists_idxs:
            artists_idxs[artist_id] = len(artists)
            artists.append(artist_name)

        field.token_id.append(artists_idxs[artist_id])
        field.token_count.append(int(plays))

Запускаем BigArtm

In [33]:
with artm.library.MasterComponent() as master:
    master.config().processors_count = 4 #Распараллеливаем на 4 ядра
    master.Reconfigure()
    perplexity_score = master.CreatePerplexityScore()
    top_tokens_score = master.CreateTopTokensScore(num_tokens=5)
    model = master.CreateModel(topics_count=200, inner_iterations_count=20)
    
    for iteration in range(0, 20):
        master.InvokeIteration(disk_path=batch_path)
        master.WaitIdle()
        model.Synchronize()
        print "Iteration#" + str(iteration) + ": Perplexity = %.3f" % perplexity_score.GetValue(model).value

    top_tokens = top_tokens_score.GetValue(model)

artm.library.Visualizers.PrintTopTokensScore(top_tokens)

Iteration#0: Perplexity = 0.000
Iteration#1: Perplexity = 58309.877
Iteration#2: Perplexity = 2072.368
Iteration#3: Perplexity = 799.147
Iteration#4: Perplexity = 573.116
Iteration#5: Perplexity = 490.276
Iteration#6: Perplexity = 448.235
Iteration#7: Perplexity = 422.624
Iteration#8: Perplexity = 405.228
Iteration#9: Perplexity = 392.446
Iteration#10: Perplexity = 382.910
Iteration#11: Perplexity = 375.811
Iteration#12: Perplexity = 370.289
Iteration#13: Perplexity = 365.733
Iteration#14: Perplexity = 361.856
Iteration#15: Perplexity = 358.584
Iteration#16: Perplexity = 355.789
Iteration#17: Perplexity = 353.382
Iteration#18: Perplexity = 351.180
Iteration#19: Perplexity = 348.982
Top tokens per topic: 
Topic#1:  placebo(0.775)  die happy(0.007)  bush(0.006)  such a surge(0.005)  初音ミク(0.005)  
Topic#2:  kreator(0.072)  sodom(0.047)  testament(0.042)  slayer(0.039)  death(0.033)  
Topic#3:  die Ärzte(0.239)  josé gonzález(0.150)  die toten hosen(0.116)  farin urlaub(0.057)  the orb(0.0